In [2]:
#########################################################################
#####                   Exercice  Aggréger des restaurants         #####

# Q : Créer 1DB restaurant, ajouter 1 collection "restaurants", la remplir avec le fichier restaurants.json.
# Indice : 25357 restaurant en tout

# Réponse : Dans une invite de commande: 
# mongoimport -d restaurant -c restaurants --drop --file "C:\Users\remik\Documents\B3\noSQL/restaurants.json"

In [13]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.restaurant
print('connexion success')

connexion success


In [17]:
# Q : La répartition des restaurants par quartier ?
# Indice : quartier => key 'borough'

restaurants = db.restaurants.find()

agregat = [
    {
      "$match": {}
   },
   {
      "$sort": {
         "borough": 1
      }
   },
   {
      "$group": {
          "_id": "$borough", "count": {"$sum": 1}
      }
   },
]


quartiersRestaurants = db.restaurants.aggregate(agregat)


for x in quartiersRestaurants:
  print(x)

{'_id': 'Queens', 'count': 5656}
{'_id': 'Staten Island', 'count': 969}
{'_id': 'Brooklyn', 'count': 6085}
{'_id': 'Bronx', 'count': 2338}
{'_id': 'Manhattan', 'count': 10258}
{'_id': 'Missing', 'count': 51}


In [18]:
# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?
# Indice : la dernière inspection : la plus récente, donc la première de la liste !
# key : "grade"

agregat = [
    {
      "$match": {"grades.0.grade":"C"}
   },
   {
      "$sort": {
         "grades": 1
      }
   },
   {
      "$group": {
          "_id": "$grades"
      }
   },
]



gradeC = db.restaurants.aggregate(agregat)

for x in gradeC:
  print(x)

{'_id': [{'date': datetime.datetime(2014, 10, 22, 0, 0), 'grade': 'C', 'score': 31}, {'date': datetime.datetime(2014, 5, 16, 0, 0), 'grade': 'A', 'score': 7}, {'date': datetime.datetime(2013, 11, 20, 0, 0), 'grade': 'A', 'score': 7}, {'date': datetime.datetime(2013, 4, 9, 0, 0), 'grade': 'B', 'score': 27}, {'date': datetime.datetime(2012, 10, 18, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2012, 5, 16, 0, 0), 'grade': 'B', 'score': 25}]}
{'_id': [{'date': datetime.datetime(2014, 6, 26, 0, 0), 'grade': 'C', 'score': 45}, {'date': datetime.datetime(2013, 11, 22, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 4, 15, 0, 0), 'grade': 'B', 'score': 15}, {'date': datetime.datetime(2012, 8, 15, 0, 0), 'grade': 'A', 'score': 11}]}
{'_id': [{'date': datetime.datetime(2013, 12, 9, 0, 0), 'grade': 'C', 'score': 32}, {'date': datetime.datetime(2013, 3, 19, 0, 0), 'grade': 'B', 'score': 27}]}
{'_id': [{'date': datetime.datetime(2014, 10, 20, 0, 0), 'grade': 'C',

In [26]:
# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?
# Indice : utiliser l'opérateur unwid

agregat = [
    {
      "$unwind": "$grades"
   },
   {
      "$group": {
          "_id": "$borough", "moyenne": {"$avg": "$grades.score"}
      }
   },
   {
      "$sort":{"moyenne":-1} 
   },
]



moyenne = db.restaurants.aggregate(agregat)

for x in moyenne:
  print(x)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


In [27]:
#########################################################################
#####                   Exercice  Aggréger des transactions         #####
# Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes :

# Réponse : dans le shell : 
# mongoimport -d restaurant -c transactions --drop --file "C:\Users\remik\Documents\B3\noSQL\transactions.json" --jsonArray

In [34]:
# Q-1. Calculer le montant total des paiements ?
# Indice : key : Payment.Total

agregat = [
    {
      "$match": {}
   },
   {
      "$group": {
          "_id": "$name","Total": {"$sum": "$Payment.Total"}
      }
   },
]



total = db.transactions.aggregate(agregat)

for x in total:
  print(x)

{'_id': None, 'Total': 1200}


In [56]:
# Q-2. Calculer le montant total par paiment ?

agregat = [
    {
      "$match": {}
   },
   {
      "$group": {
          "_id": "$Name","Total": {"$sum": "$Payment.Total"}
      }
   },
   {
      "$sort": {
         "name": 1
      }
   },
]



paiementsTotaux = db.transactions.aggregate(agregat)

for x in paiementsTotaux:
  print(x)

{'_id': 'Dylan', 'Total': 0}
{'_id': 'John', 'Total': 400}
{'_id': 'Tom', 'Total': 400}
{'_id': 'Margaret', 'Total': 400}


In [32]:
# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

agregat = [
    {
      "$match": {"Payment.Success":True}
   },
   {
      "$group": {
          "_id": "$Payment.Type", "Total":{"$sum":"$Payment.Total"}
      }
   }
]



reponse = db.transactions.aggregate(agregat)

for x in reponse:
  print(x)

{'_id': 'Credit-Card', 'Total': 800}
{'_id': 'Debit-Card', 'Total': 400}


In [54]:
# Q-4. Trouver l'Id le plus élevé.

agregat = [
    {
      "$match": {}
   },
   {
      "$group": {
          "_id": "$name","Max id": {"$max": "$Id"}
      }
   },
]



maxId = db.transactions.aggregate(agregat)

for x in maxId:
  print(x)

{'_id': None, 'Max id': 103}


In [53]:
# Q-5. Find the max price (Transaction.price) ?

agregat = [
    {
      "$match": {}
   },
   {
      "$group": {
          "_id": "$name","Max price": {"$max": "$Transaction.price"}
      }
   },
]



maxPrice = db.transactions.aggregate(agregat)

for x in maxPrice:
  print(x)

{'_id': None, 'Max price': [200, 210]}


In [ ]:
# Q : Age unique => distinct ?
# db.people.distinct("age")
# S : [ 20, 35, 60 ]
# Q : Peut-on faire un distinct sur plusieurs key ?
# db.people.distinct("name", "age")
# R : Ce n'est pas possible de faire un distinct sur plusieurs key => Il faut passer par map reduce !